In [5]:
import os
import pandas as pd
import numpy as np
from scipy import misc
from sklearn.cross_validation import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
np.random.seed(1337)

In [6]:
img_path = "/root/sharedfolder/clean/"
legend = pd.read_csv("/root/sharedfolder/legend/legend-clean.csv")

In [7]:
batch_size = 128
nb_classes = 8
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 96, 96
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

In [8]:
legend['emotion'] = legend['emotion'].str.lower()

In [9]:
# 'neutral' = 0
# 'happiness' = 1
# 'surprise' = 2
# 'anger' = 3
# 'sadness' = 4
# 'contempt' = 5
# 'disgust' = 6
# 'fear' = 7
legend.replace('neutral', 0, inplace=True)
legend.replace('happiness', 1, inplace=True)
legend.replace('surprise', 2, inplace=True)
legend.replace('anger', 3, inplace=True)
legend.replace('sadness', 4, inplace=True)
legend.replace('contempt', 5, inplace=True)
legend.replace('disgust', 6, inplace=True)
legend.replace('fear', 7, inplace=True)

In [10]:
# convert images to numpy.ndarray

images = np.empty(shape=[len(legend), img_rows, img_cols])
for i in range(0,len(legend)):
    temp = img_path + legend.iloc[i,0]
    img = misc.imread(temp, flatten=True,)
    img = misc.imresize(img, (img_rows, img_cols))
    images[i] = img

In [11]:
emotions = legend['emotion']

In [12]:
# the data, shuffled and split between train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, emotions, test_size=0.25)

In [14]:
X_train.shape

(10260, 96, 96)

In [15]:
X_test.shape

(3421, 96, 96)

In [16]:
img_rows, img_cols = X_train.shape[1], X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)

In [17]:
X_train.shape

(10260, 1, 96, 96)

In [18]:
X_test.shape

(3421, 1, 96, 96)

In [19]:
input_shape = (1, img_rows, img_cols)

In [20]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('X_train shape:', (10260, 1, 96, 96))
(10260, 'train samples')
(3421, 'test samples')
Train on 10260 samples, validate on 3421 samples
Epoch 1/12
10260/10260 [==============================] - 614s - loss: 1.6608 - acc: 0.4975 - val_loss: 1.0664 - val_acc: 0.4984
Epoch 2/12
10260/10260 [==============================] - 677s - loss: 0.9995 - acc: 0.5986 - val_loss: 0.9644 - val_acc: 0.6346
Epoch 3/12
10260/10260 [==============================] - 696s - loss: 0.8304 - acc: 0.6981 - val_loss: 0.8806 - val_acc: 0.5960
Epoch 4/12
10260/10260 [==============================] - 690s - loss: 0.7322 - acc: 0.7449 - val_loss: 0.7133 - val_acc: 0.7632
Epoch 5/12
10260/10260 [==============================] - 655s - loss: 0.6536 - acc: 0.7739 - val_loss: 0.7540 - val_acc: 0.7135
Epoch 6/12
10260/10260 [==============================] - 683s - loss: 0.6042 - acc: 0.7919 - val_loss: 0.6108 - val_acc: 0.7933
Epoch 7/12
10260/10260 [==============================] - 705s - loss: 0.5582 - acc: 0.804